# EDA Step 6: 핵심 인사이트 정리

**목적**: EDA의 최종 산출물 - "데이터가 말하는 바"를 정리

---

## 📊 데이터 개요

| 데이터 | 규모 | 설명 |
|--------|------|------|
| books.csv | 149,570개 | 책 메타데이터 (10개 컬럼) |
| users.csv | 68,092명 | 사용자 메타데이터 (3개 컬럼) |
| train_ratings.csv | 306,795건 | 학습용 평점 데이터 |
| test_ratings.csv | 76,699건 | 예측 대상 데이터 |

### 핵심 수치
- **Sparsity**: 99.996% (매우 희소)
- **User당 평균 rating**: 5.13개 (중앙값 1개)
- **Item당 평균 rating**: 2.36개 (중앙값 1개)
- **Rating 평균**: 7.07 (중앙값 8, 고평점 편향)

---

## 🚨 핵심 문제점 (Challenge)

### 1. 극심한 Long-Tail 분포

| 구분 | 1개 rating만 보유 | 의미 |
|------|-------------------|------|
| **User** | 57.9% (34,617명) | 과반수 User가 CF 불가 |
| **Item** | 68.1% (88,392개) | 2/3 Item이 CF 불가 |

→ **순수 Collaborative Filtering만으로는 한계**

### 2. Cold Start 문제

| 구분 | Test 내 비율 | 대응 가능성 |
|------|-------------|-------------|
| Cold User | 11.3% (8,677건) | User 메타데이터 활용 |
| Cold Item | **25.8%** (19,793건) | Author로 62% 커버 가능 |
| 둘 다 Cold | 2.3% (1,734건) | 전체 평균 + 메타데이터 |

→ **Cold Item이 핵심 문제, Author 정보로 상당 부분 해결 가능**

### 3. 결측치 현황

| 데이터 | 컬럼 | 결측률 | 대응 방안 |
|--------|------|--------|----------|
| books | language | 44.95% | 'unknown' 또는 'en' 대체 |
| books | category | 46.03% | 'unknown' 대체 |
| books | summary | 44.95% | 텍스트 피처 사용 시 주의 |
| users | age | 40.88% | 중앙값(34) 또는 그룹 평균 대체 |

---

## 💡 핵심 발견 (Key Insights)

### 1. Author가 가장 강력한 피처 🔥

| 지표 | 수치 | 의미 |
|------|------|------|
| Author 평균 예측 RMSE | 2.0953 | 전체 평균(2.4332) 대비 **13.89% 개선** |
| 작가별 Rating 차이 | 최대 2점+ | J.K. Rowling 8.68 vs Danielle Steel 6.51 |
| Cold Item 중 Author 활용 가능 | 62.0% | 12,268개 Cold Item 커버 가능 |

**활용 방안**:
- Author bias term 모델에 추가
- Cold Item → 같은 Author의 평균 rating으로 예측
- Author embedding 학습

### 2. User/Item Bias가 매우 큼

| 구분 | 평균 Rating | 차이 |
|------|------------|------|
| 까다로운 User (50%+ 저평점) | 3.51 | |
| 후한 User (5%- 저평점) | 8.04 | **4.53점 차이** |
| 저평가 Item | 낮음 | |
| 고평가 Item | 높음 | |

**활용 방안**:
- User bias, Item bias term 필수 포함
- SVD++, FM 등 bias term 있는 모델 활용

### 3. Heavy User 특성

| 발견 | 내용 |
|------|------|
| 비율 | 상위 1% (601명)가 30.5% rating 생성 |
| 평점 경향 | 평균 7.30 (Normal 6.97보다 0.33점 높음) |
| Item 선호 | **비인기 Item 더 많이 평가** (인기Item 24.3% vs Normal 36.2%) |
| 취향 유사도 | 매우 낮음 (Jaccard 0.0024) |

**시사점**: Heavy User는 "후한 사람"이 아니라 "다양하게 탐색하는 사람"
→ 샘플링 조정보다는 그대로 활용

### 4. Age × Category 강한 상관

| 연령대 | Juvenile Fiction | Fiction |
|--------|------------------|--------|
| ~20 | **37.3%** | 50.9% |
| 50+ | 3.5% | **83.4%** |

**활용 방안**: Age group × Category 교차 피처 생성

### 5. 고전 vs 최신 패턴

| 구분 | 평균 Rating | 10점 비율 |
|------|------------|----------|
| 고전 (1900-1979) | **7.30** | **19.4%** |
| 최신 (2000-2006) | 7.13 | 13.5% |

**시사점**: 고전은 "검증된 명작"으로 평점이 높음
→ 출판연도 피처 활용 가치 있음

---

## 📈 Train vs Test 특성 비교

| 항목 | Train | Test | 해석 |
|------|-------|------|------|
| User Activity 중앙값 | 1개 | **5개** | Test User가 더 활발 |
| Item Popularity 중앙값 | 1개 | **3개** | Test Item이 더 인기 |
| Category 분포 | - | - | 거의 동일 |
| 출판연도 분포 | - | - | 거의 동일 |
| Warm User 평균 Rating | - | 7.08 | 전체 평균과 유사 |
| Warm Item 평균 Rating | - | 7.14 | 전체 평균보다 약간 높음 |

**핵심 시사점**: 
- Test는 **활발한 User + 인기 Item** 위주로 구성
- Cold Start 외에는 예측이 상대적으로 쉬운 케이스
- Category, 출판연도 분포는 Train과 동일 → 분포 shift 없음

---

## 🛠 피처 엔지니어링 제안

### 필수 피처 (High Priority)

| 피처 | 설명 | 기대 효과 |
|------|------|----------|
| **user_bias** | 각 User의 평균 rating - 전체 평균 | User 개인 성향 반영 |
| **item_bias** | 각 Item의 평균 rating - 전체 평균 | Item 품질 반영 |
| **author_mean_rating** | 각 Author의 평균 rating | **RMSE 14% 개선** |
| **user_rating_count** | User의 rating 수 (log 변환) | Activity level |
| **item_rating_count** | Item의 rating 수 (log 변환) | Popularity level |

### 권장 피처 (Medium Priority)

| 피처 | 설명 | 기대 효과 |
|------|------|----------|
| **category_main** | 첫 번째 카테고리 | Content 정보 |
| **publisher** | 출판사 | 출판사별 품질 차이 |
| **year_of_publication** | 출판연도 | 고전 효과 반영 |
| **is_classic** | 1980년 이전 여부 | 고전 가산점 |
| **age_group** | 연령대 구간화 | User 특성 |
| **country** | 국가 | 국가별 평점 경향 |

### 교차 피처 (Optional)

| 피처 | 설명 |
|------|------|
| **age_group × category** | 연령-장르 선호도 |
| **country × category** | 국가-장르 선호도 |
| **user_activity × item_popularity** | 활발한 User의 비인기 Item 선호 |

---

## 🎯 모델링 전략 제안

### Baseline 모델

```
예측값 = 전체 평균 + User bias + Item bias
예상 RMSE: ~2.3 수준
```

### 권장 모델 아키텍처

| 모델 | 장점 | Cold Start 대응 |
|------|------|----------------|
| **FM (Factorization Machine)** | Sparse 데이터에 강함, 피처 교차 자동 학습 | 메타데이터 활용 가능 |
| **DeepFM** | FM + DNN, 비선형 패턴 학습 | 메타데이터 활용 가능 |
| **SVD++ with side info** | User/Item bias 포함, 검증된 방법 | 제한적 |
| **NCF + Content** | Neural CF + Content 정보 결합 | 메타데이터 활용 가능 |

### Cold Start 전략

| Cold 유형 | 전략 | 커버리지 |
|----------|------|----------|
| **Cold User** | User 메타데이터(age, country) 기반 예측 | 11.3% |
| **Cold Item (Author 있음)** | Author 평균 rating | **62% of Cold Item** |
| **Cold Item (Author 없음)** | Category/Publisher 평균 또는 전체 평균 | 38% of Cold Item |
| **둘 다 Cold** | 전체 평균 (7.07) + 메타데이터 보정 | 2.3% |

### 앙상블 전략

```
1. CF 기반 모델 (SVD, ALS 등) → Warm User-Item에 강함
2. Content 기반 모델 (FM, DeepFM 등) → Cold Start에 강함
3. 가중 평균 또는 Stacking으로 결합
```

---

## ⚠️ 주의사항

### 1. 데이터 누수 방지
- User/Item 통계 (평균, count 등)는 **Train에서만** 계산
- Test에 적용 시 Train 통계 사용
- Cold User/Item은 통계값 없음 → 전체 평균 또는 0으로 대체

### 2. 결측치 처리
- **age**: 중앙값(34) 또는 그룹별 평균으로 대체
- **category**: 'unknown' 카테고리로 대체
- **language**: 'en' (최빈값) 또는 'unknown'으로 대체
- **author_mean_rating**: Cold Author는 전체 평균으로 대체

### 3. 평가 시 고려사항
- Test 내 Cold Start 비율이 높음 (Item 25.8%)
- Validation 시 Cold Start 시나리오 별도 평가 권장
- RMSE 외에 Cold/Warm 케이스 별도 분석

### 4. 이상치 처리
- **year_of_publication**: 1376, 1378년 등 3건 → 중앙값(1996)으로 대체
- **Heavy User**: 제거하지 않고 그대로 사용 (다양한 Item 평가자)

---

## 📋 EDA 체크리스트 (완료)

| 단계 | 항목 | 상태 |
|------|------|------|
| **Step 1** | 데이터 구조 파악 | ✅ |
| **Step 2** | 데이터 품질 점검 | ✅ |
| **Step 3** | 단일 변수 탐색 | ✅ |
| **Step 4** | 변수 간 관계 탐색 | ✅ |
| **Step 5** | 패턴 탐색 및 가설 형성 | ✅ |
| **Step 6** | 핵심 인사이트 정리 | ✅ |

---

## 🎯 Next Steps (모델링)

1. **피처 엔지니어링 코드 작성**
   - User/Item 통계 피처
   - Author 평균 rating 피처
   - 범주형 변수 인코딩

2. **Baseline 모델 구축**
   - Global Mean + User Bias + Item Bias
   - 예상 RMSE: ~2.3

3. **본격 모델링**
   - FM/DeepFM 등 피처 기반 모델
   - Cold Start 케이스 별도 처리

4. **앙상블**
   - 다양한 모델 결합
   - Cold/Warm 케이스별 가중치 조정

---

## 📊 핵심 수치 요약 (Quick Reference)

```
=== 데이터 규모 ===
Train: 306,795건 (59,803 users × 129,777 items)
Test: 76,699건 (26,167 users × 52,000 items)
Sparsity: 99.996%

=== Rating 통계 ===
평균: 7.07
중앙값: 8
표준편차: 2.43

=== Long-Tail ===
1-rating User: 57.9%
1-rating Item: 68.1%
Heavy User (top 1%): 30.5% of ratings

=== Cold Start (Test) ===
Cold User: 11.3%
Cold Item: 25.8%
Cold Item with known Author: 62.0%

=== 결측치 ===
books.language: 44.95%
books.category: 46.03%
users.age: 40.88%

=== Baseline 성능 ===
전체 평균 RMSE: 2.4332
Author 평균 RMSE: 2.0953 (13.89% 개선)
```

In [ ]:
# 피처 엔지니어링 코드 템플릿 (참고용)

import pandas as pd
import numpy as np

def create_features(train, test, users, books):
    """
    EDA 결과를 바탕으로 피처 생성
    """
    # 1. User 통계 (Train에서만 계산)
    user_stats = train.groupby('user_id').agg(
        user_rating_count=('rating', 'count'),
        user_rating_mean=('rating', 'mean'),
        user_rating_std=('rating', 'std')
    ).reset_index()
    
    # 2. Item 통계 (Train에서만 계산)
    item_stats = train.groupby('isbn').agg(
        item_rating_count=('rating', 'count'),
        item_rating_mean=('rating', 'mean'),
        item_rating_std=('rating', 'std')
    ).reset_index()
    
    # 3. Author 통계 (Train에서만 계산)
    train_with_author = train.merge(books[['isbn', 'book_author']], on='isbn', how='left')
    author_stats = train_with_author.groupby('book_author').agg(
        author_rating_mean=('rating', 'mean'),
        author_rating_count=('rating', 'count')
    ).reset_index()
    
    # 4. 전체 평균
    global_mean = train['rating'].mean()
    
    # 5. Bias 계산
    user_stats['user_bias'] = user_stats['user_rating_mean'] - global_mean
    item_stats['item_bias'] = item_stats['item_rating_mean'] - global_mean
    author_stats['author_bias'] = author_stats['author_rating_mean'] - global_mean
    
    # 6. 데이터 병합
    # Train
    train_fe = train.merge(user_stats, on='user_id', how='left')
    train_fe = train_fe.merge(item_stats, on='isbn', how='left')
    train_fe = train_fe.merge(users, on='user_id', how='left')
    train_fe = train_fe.merge(books, on='isbn', how='left')
    train_fe = train_fe.merge(author_stats, on='book_author', how='left')
    
    # Test (Cold Start 처리 포함)
    test_fe = test.merge(user_stats, on='user_id', how='left')
    test_fe = test_fe.merge(item_stats, on='isbn', how='left')
    test_fe = test_fe.merge(users, on='user_id', how='left')
    test_fe = test_fe.merge(books, on='isbn', how='left')
    test_fe = test_fe.merge(author_stats, on='book_author', how='left')
    
    # 7. Cold Start 처리 (결측값 → 전체 평균/0으로 대체)
    fill_values = {
        'user_rating_mean': global_mean,
        'item_rating_mean': global_mean,
        'author_rating_mean': global_mean,
        'user_bias': 0,
        'item_bias': 0,
        'author_bias': 0,
        'user_rating_count': 0,
        'item_rating_count': 0,
        'user_rating_std': train['rating'].std(),
        'item_rating_std': train['rating'].std(),
    }
    
    for col, val in fill_values.items():
        if col in test_fe.columns:
            test_fe[col] = test_fe[col].fillna(val)
    
    return train_fe, test_fe, global_mean

print("피처 엔지니어링 함수 정의 완료")
print("실제 사용 시 train, test, users, books 데이터프레임을 인자로 전달하세요.")

In [ ]:
# Simple Baseline 모델 (참고용)

def simple_baseline_predict(test_fe, global_mean):
    """
    Simple Baseline: Global Mean + User Bias + Item Bias + Author Bias
    """
    predictions = (
        global_mean 
        + test_fe['user_bias'].fillna(0) 
        + test_fe['item_bias'].fillna(0)
        + test_fe['author_bias'].fillna(0) * 0.5  # Author bias는 가중치 조정
    )
    
    # Clipping (1-10 범위)
    predictions = predictions.clip(1, 10)
    
    return predictions

print("Baseline 예측 함수 정의 완료")

---

## 🏁 EDA 완료!

이 문서를 참고하여 모델링을 진행하세요.

**핵심 포인트 3가지**:
1. **Author 피처 필수** - 단독으로 14% RMSE 개선
2. **User/Item Bias 필수** - 개인 성향 차이가 매우 큼
3. **Cold Start 전략** - Author로 62% 커버, 나머지는 메타데이터 활용